In [ ]:
import pandas as pd
import numpy as np
import mplfinance as mpf
import os
import pandas_ta as ta

import yfinance as yf
import matplotlib.pyplot as plt

# Run snp500 listupdate to get the list of latest snp500 tickers: http://localhost:8888/lab/tree/snp500listupdate.ipynb

In [ ]:
#st_df=pd.read_csv("/home/thakur/stock_information/sectoretf.csv") #more sectors
st_df=pd.read_csv("/home/thakur/stock_information/sectors.csv")  #11 sectors
st_list=st_df["TICKER"].to_list()
# st_df

In [ ]:
#data=yf.download("XLE",period='20d')
# sector_list=st_df['SECTOR'].to_list()
# sector_list

In [ ]:
#Download all tickers
save="/home/thakur/test/sector/"
print(f"\nDowloading {st_list} data from yahoo finance...\n")
data = yf.download(
tickers=st_list,
period='52wk',
threads=True,
group_by='ticker',
rounding=True)

#save to csv filesb
print(f"Saving to {save}...\n")
for i in st_list:
    df=data[i]
    df.to_csv(save+i+".csv")
    print(f"{i}.csv created !")
    #print(f"{i} head:\n")
    #print(df.head(1))


In [ ]:
def change_index(df):
    """Begins the index of df from 1"""
    df.index=np.arange(1,len(df.index)+1)
    return df

#create yfinance portfolio sample
def yahoo_portfolio(df,save_name):
    """
    returns the yfinance compatable portfolio template for given df name of save file
    """
    cols=['Symbol','Current Price','Date', 'Time', 'Change', 'Open', 'High', 'Low', 'Volume', 'Trade Date',
          'Purchase Price','Quantity','Commission','High Limit','Low Limit','Comment']

    sym=list(df.TICK)
    dic_val=[sym]+14*[None]            #14 None
    my_dict=dict(zip(cols,dic_val))    #dictionary
    my_df=pd.DataFrame.from_dict(my_dict)
    my_df.to_csv(save_name+".csv")
    print(f"Created the yahoo template: {save_name}.csv from data frame !")
    
def plot_with_mpl(tick,root,day=60,sty='charles',typ='candle'):
    "returns the candle stick of a stock tick for the given root file."

    df=pd.read_csv(root+tick+".csv").tail(day)
    df.Volume=df.Volume/10**5              #100k volume
    df.Date=pd.to_datetime(df.Date)
    df.set_index('Date',inplace=True)
    fig,axes=mpf.plot(df,type=typ,mav=(5,10),volume=True,tight_layout=False,figratio=(72,36),figscale=1,
                    returnfig=True,style=sty)
    axes[0].legend(["5-SMA","10-SMA"],loc='upper center')
    #axes[0].legend(loc='upper left')
    axes[0].set_title(tick)
    mpf.show()

    
def print_info(c,sector,industry,name,symbol,cp):
    print(f"\n\n\nCOUNT   : {c+1}")
    print(f"SECTOR  : {sector}\nINDUSTRY: {industry}\nNAME    : {name}\nSYMBOL  : {symbol}\nPRICE   : $ {cp}\n\n")
    


In [ ]:
#Reading the files
ticker_list=[]           #store ticker
current_price=[]         #current price

HH=[]                    #higher high
HL=[]                    #higher low
LH=[]                    #lower high
LL=[]                    #lower low
HC=[]                    #higher close
rsi=[]                   #rsi list
sma_5=[]                 #sma_5 list
sma_21=[]                 #sma_21 list
sma_50=[]                 #sma_50 list
sma_200=[]                 #sma_200 list
vol=[]                    #vol today
avgvol=[]                 #avg volume
pt_change=[]             #% change
atr=[]                   #atr
sec_list=[]             #sector
ind_list=[]             #industry
d=3                     #days for hh etc.
j=1                     #counter
fil="sector"
csv_source="/home/thakur/test/sector/"
for f in st_list:#[:5]:    #read these files
    print(f"{j}/{len(st_list)} {fil} : {f}\n")
    j+=1

    #print("source dir\t",csv_source)
    df=pd.read_csv(csv_source+f+".csv")
    df.fillna(0,inplace=True)

    #print the tail
    #print(f"Tail of {i}\n {df.tail(1)}")

    #ticker
    ticker_list.append(f)

    #current price 
    cp=round(df.tail(1)['Close'].values[0],2)
    #print(f"Current Price: {cp}")
    current_price.append(cp)

    #higher high
    hh=df.tail(d)['High'].is_monotonic_increasing
    #print(f"HH: {hh}\n")
    HH.append(hh)

    ##higher low
    hl=df.tail(d)['Low'].is_monotonic_increasing
    #print(f"HL: {hl}\n")
    HL.append(hl)

    #lower high
    lh=df.tail(d)['High'].is_monotonic_decreasing
    #print(f"LH: {lh}\n")
    LH.append(lh)

    ##lower low
    ll=df.tail(d)['Low'].is_monotonic_decreasing
    #print(f"LL: {ll}\n")
    LL.append(ll)

    #higher close
    hc=df.tail(d)['Close'].is_monotonic_increasing
    #print(f"HC: {hc}\n")
    HC.append(hc)

    #rsi
    rs=df.ta.rsi()[-1:].round(2).values[0]
    #print(f"rs\t{rs}")
    rsi.append(rs)

    #sma5
    sma5=round(df.tail(5)['Close'].mean(),2)
    #print(f"sma5: {sma5}")
    sma_5.append(sma5)

    #sma21
    sma21=round(df.tail(21)['Close'].mean(),2)
    #print(f"sma21: {sma21}")
    sma_21.append(sma21)

    #sma50
    sma50=round(df.tail(50)['Close'].mean(),2)
    #print(f"sma50: {sma50}")
    sma_50.append(sma50)

    #sma200
    sma200=round(df.tail(200)['Close'].mean(),2)
    #print(f"sma200: {sma200}")
    sma_200.append(sma200)

    #last volume
    v=(df.tail(1)['Volume'].values[0])/10**6
    v=round(v,2)
    vol.append(v)
    #print(f"today's volume {v} thousands")

    #last volume
    av=(df.tail(d)['Volume'].mean())/10**5
    av=round(av,2)
    avgvol.append(av)
    #print(f"today's volume {av} thousands")

    #pct change
    pct=round((df.tail(2)['Close'].pct_change().values[1])*100,2)
    #print(f"%Change:\n {pct}")
    #pct=2
    pt_change.append(pct)
    #pct_change.append(pct)
    #print(f"%Change: {pct}")

    #atr value
    df_20=df.tail(24) #for 10 day avg atr
    #at=round(ta.atr(df_20['High'],df_20['Low'],df_20['Close']).values[-1],2)
    at=round(ta.atr(df_20['High'],df_20['Low'],df_20['Close']).mean(),2)
    #print(f"atr: {at}")
    atr.append(at)
        
        
        
        
#create the data-frame
temp_df=pd.DataFrame({"TICKER":ticker_list,
                          "CP":current_price,
                          "HH":HH,
                          "HL":HL,
                          "LH":LH,
                          "LL":LL,
                          "HC":HC,
                          "RSI":rsi,
                          "SMA5":sma_5,
                          "SMA21":sma_21,
                          "SMA50":sma_50,
                          "SMA200":sma_200,
                          "VOL":vol,
                          "AVGVOL":avgvol,
                          "%CHG":pt_change,
                          "AVG10ATR":atr,
                         })
temp_df.to_csv('/home/thakur/test/combined/all_sectors.csv',index=False)

In [ ]:
temp_df["SECTOR"]=st_df["SECTOR"];temp_df["SUBINDUSTRY"]="N/A";temp_df["NAME"]="N/A"
#temp_df=temp_df[(temp_df['HH']==True)&(temp_df['HL']==True)]
temp_df=temp_df.sort_values(by=['RSI'],ascending=False)
temp_df=change_index(temp_df)
#temp_df.set_index('TICKER',inplace=True)
temp_df.head(25)

In [ ]:
plot_df=temp_df[['TICKER','RSI','VOL','%CHG','AVG10ATR']]
#plot_df.loc[:,'%CHG']=plot_df.loc[:,'%CHG'].multiply(10)
#plot_df.reset_index(inplace=True)
plot_df

In [ ]:
columns=plot_df.columns.to_list()
x=columns[0];y=columns[1:]

import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
today=datetime.now().strftime("%m/%d/%Y")
#sns.set_style("ticks",{'axes.grid' : True})

# sns.set_style("whitegrid", {
#     "ytick.major.size": 0.1,
#     "ytick.minor.size": 0.05,
#     'grid.linestyle': '--'
#  })

# Apply the default theme
sns.set_theme()
#sns.bar(plot_df)
# sns.barplot(data=plot_df,x='TICKER',y='RSI')
# sns.barplot(data=plot_df,x='TICKER',y='VOL')
f, axs = plt.subplots(2, 2, figsize=(15, 10), gridspec_kw=dict(width_ratios=[4, 4]))
cols=axs.flat
for i,j in enumerate(y):
    g=sns.barplot(data=plot_df,x=x,y=y[i],ax=cols[i])
    #g.semilogy()
f.suptitle(f"SECTOR {today}")   
f.tight_layout()
f.savefig("sector.pdf")


In [ ]:
#all snp data
pd.set_option('display.max_columns', None);pd.set_option('display.max_rows', None)
# select_cols_snp=['TICKER', 'CP', 'EMA8','RSI',
#        'SMA5', 'SMA10', 'SMA21', 'SMA50', 'SMA200', 'VOL', 'AVGVOL5', 'CHG%',
#        'ATR14']
select_cols_w=['SYMBOL', 'NAME', 'SECTOR', 'SUBINDUSTRY']

df_snp=pd.read_csv("/home/thakur/test/combined/all_snp.csv")#[select_cols_snp]
df_snp_w_sector=pd.read_csv("/home/thakur/stock_information/snp500finalwvol.csv")[select_cols_w]
df_snp_w_sector.rename(columns={"SYMBOL":"TICKER"},inplace=True)
df_snp.head()

In [ ]:
df_snp_w_sector.head()

In [ ]:
df_snp_w_sector.shape,df_snp.shape, 

In [ ]:
for i in ['NAME','SECTOR','SUBINDUSTRY']:df_snp[i]=df_snp_w_sector[i]
 
#df_snp['NAME']=df_snp_w_sector['NAME'];df_snp['SECTOR']=df_snp_w_sector['SECTOR'];df_snp['SUBINDUSTRY']=df_snp_w_sector['SUBINDUSTRY']

In [ ]:
#getting the type of the companay mega, large, medium, small, micro, nano
import os
root="/home/thakur/test/"
group=['mega','large','medium','small','micro','nano']
dirs=[root+i+"/" for i in group]
#check_list=['JPM','MOS','HAS','HBIO']
check_list=list(df_snp.TICKER)
mega_list=os.listdir(dirs[0])
large_list=os.listdir(dirs[1])
medium_list=os.listdir(dirs[2])
small_list=os.listdir(dirs[3])
micro_list=os.listdir(dirs[4])
nano_list=os.listdir(dirs[5])

category=[]
for i in check_list:
    file=i+".csv"
    if file in mega_list:
        #print(f"{i}: mega")
        #category.append([i,"mega"])
        category.append("mega")
    elif file in large_list:
        #print(f"{i}: large")
        #category.append([i,"large"])
        category.append("large")
    elif file in medium_list:
        #print(f"{i}: medium")
        category.append("medium")
        #category.append([i,"medium"])
    elif file in small_list:
        #print(f"{i}: small")
        category.append("small")
        #category.append([i,"small"])
    elif file in micro_list:
        #print(f"{i}: micro")
        category.append("micro")                
        #category.append([i,"micro"])
    elif file in nano_list:
        #print(f"{i}: nano")
        category.append("nano")
        #category.append([i,"nano"])
    else:
        #print(f"{i}: nano")
        category.append("N/A")
#nano_list

# for i in group:
#     dirr=root+i+"/"
#     ticker_list=os.listdir(dirr)
#     print(f'ticker list {ticker_list}')

In [ ]:
len(category)

In [ ]:
df_snp['CATEGORY']=category
df_snp.head()

In [ ]:
df_snp.loc[df_snp['CATEGORY']=='N/A']

In [ ]:
#getting the type of the companay mega, large, medium, small, micro, nano
import os
root="/home/thakur/test/"
group=['mega','large','medium','small','micro','nano']
dirs=[root+i+"/" for i in group]
#check_list=['JPM','MOS','HAS','HBIO']
check_list=list(df_snp.TICKER)
mega_list=os.listdir(dirs[0])
large_list=os.listdir(dirs[1])
medium_list=os.listdir(dirs[2])
small_list=os.listdir(dirs[3])
micro_list=os.listdir(dirs[4])
nano_list=os.listdir(dirs[5])

for i in check_list:
    file=i+".csv"
    if file in mega_list:
        print(f"{i}: mega")
    elif file in large_list:
        print(f"{i}: large")
    elif file in medium_list:
        print(f"{i}: medium")
    elif file in small_list:
        print(f"{i}: small")
    elif file in micro_list:
        print(f"{i}: micro")
    elif file in nano_list:
        print(f"{i}: nano")
#nano_list

# for i in group:
#     dirr=root+i+"/"
#     ticker_list=os.listdir(dirr)
#     print(f'ticker list {ticker_list}')

In [ ]:
#getting the type of the companay mega, large, medium, small, micro, nano
import os
root="/home/thakur/test/"
group=['mega','large','medium','small','micro','nano']
check_tick='PLTR'

# for c,i in enumerate(group):
#     path=path+group[c]
#     file=check_tick+'.csv'
#     if os.path.exists(file):
#         print(f"File: {file} exista at {path}")
#         break
#     #if check_tick+".csv' 
def get_categorty(file):
    # for c,i in enumerate(group):
    #     path=path+group[c]
    #     print(f"path: {path}")
    #     file=check_tick+'.csv'
    path=root+group[0]
    if os.path.exists(file):
        print(f"File: {file} exista at {path}")
        path="/home/thakur/test/"
        print(f"path: {path}")
        return group[0]
    
    path=root+group[1]
    if os.path.exists(file):
        print(f"File: {file} exista at {path}")
        path="/home/thakur/test/"
        print(f"path: {path}")
        return group[1]
    
    path=root+group[2]
    if os.path.exists(file):
        print(f"File: {file} exista at {path}")
        path="/home/thakur/test/"
        print(f"path: {path}")
        return group[2]
    
    path=root+group[3]
    if os.path.exists(file):
        print(f"File: {file} exista at {path}")
        path="/home/thakur/test/"
        print(f"path: {path}")
        return group[3]
    
    path=root+group[4]
    if os.path.exists(file):
        print(f"File: {file} exista at {path}")
        path="/home/thakur/test/"
        print(f"path: {path}")
        return group[4]
    
    path=root+group[5]
    if os.path.exists(file):
        print(f"File: {file} exista at {path}")
        path="/home/thakur/test/"
        print(f"path: {path}")
        return group[5]
        # elif os.path.exists(file):
        #     print(f"File: {file} exista at {path}")
        #     path="/home/thakur/test/"
        #     print(f"path: {path}")
        #     return group[c]
        # else:
        #     continue


# def get_categorty(check_tick,path):
#     for c,i in enumerate(group):
#         path=path+group[c]
#         print(f"path: {path}")
#         file=check_tick+'.csv'
#         if os.path.exists(file):
#             print(f"File: {file} exista at {path}")
#             path="/home/thakur/test/"
#             print(f"path: {path}")
#             return group[c]
#         else:
#             continue
get_categorty('AMD')

In [ ]:
total=list(df_snp['TICKER'])
for k in total:
    get_categorty(k,path)

In [ ]:
rsi_order=list(temp_df.SECTOR)
rsi_order

In [ ]:
#df_snp.loc['Health Care']
choice='Energy'
kind='CP'
temp_df=df_snp[df_snp['SECTOR']==choice]
h_test=((temp_df['HH']==True)&(temp_df['HL']==True) & (temp_df['HC']==True))
temp_df=temp_df[h_test]
temp_df=temp_df.sort_values(by=[kind],ascending=True)
temp_df=change_index(temp_df)
temp_df

In [ ]:
csv_source='/home/thakur/test/snp500/'
for c,i in enumerate(temp_df['TICKER'].tolist()):
    sector=temp_df.loc[c+1,"SECTOR"];industry=temp_df.loc[c+1,"SUBINDUSTRY"];cp=temp_df.loc[c+1,"CP"];name=temp_df.loc[c+1,"NAME"]

    print_info(c,sector,industry,name,i,cp)
    plot_with_mpl(i,csv_source,20,'charles')

In [ ]:
def plot_with_mpl(tick,root,day=60,sty='yahoo',typ='candle'):
    "returns the candle stick of a stock tick for the given root file."

    df=pd.read_csv(root+tick+".csv").tail(day)
    df.Volume=df.Volume/10**5              #100k volume
    df.Date=pd.to_datetime(df.Date)
    df.set_index('Date',inplace=True)
    fig,axes=mpf.plot(df,type=typ,mav=(5,10),volume=True,tight_layout=False,figratio=(72,36),figscale=1,
                    returnfig=True,style=sty)
    axes[0].legend(["5-SMA","10-SMA"],loc='upper center')
    #axes[0].legend(loc='upper left')
    axes[0].set_title(tick)
    mpf.show()

In [ ]:
temp_df.query('HH & HL & HC').sort_values(by='CP')
temp_df.plot(x=temp_df.TICKER,y=temp_df.RSI,kind='bar')

In [ ]:
import math
root='/home/thakur/test/snp500/'
day=50
cols=math.ceil(temp_df.shape[0]/3)


fig,axs=plt.figure(3,cols)

for tick in temp_df['TICKER'].to_list():
    df=pd.read_csv(root+tick+".csv").tail(day)
    df.Volume=df.Volume/10**5                             #100k volume
    df.Date=pd.to_datetime(df.Date)
    df.set_index('Date',inplace=True)
    df['Adj Close'].plot()
    print(f"{df.head()}")

In [ ]:
import math
math.ceil(temp_df.shape[0]/3)


In [ ]:

#return a df for given sector components
def sector_df(sector):
    return df_snp.loc("SECTOR"==sector)


In [ ]:
energy=sector_df("Energy")
energy

In [ ]:
sector=list(df_snp.SECTOR.unique());subindustry=list(df_snp.SUBINDUSTRY.unique())
sector

In [ ]:
tick='XLF'
df=pd.read_csv(f'/home/thakur/test/sector/{tick}.csv').tail(20)
df['Date']=df['Date'].apply(lambda x:x.split()[0])
df['Volume']=df['Volume'].divide(10**6).round(2)
#df['Date']=pd.to_datetime(df['Date'])
#df=df.set_index('Date')
df.head()

In [ ]:
# #plot_df.plot(kind='bar')
# # Import seaborn
# import seaborn as sns

# # Apply the default theme
# sns.set_theme()
# #sns.bar(plot_df)
# # sns.barplot(data=plot_df,x='TICKER',y='RSI')
# # sns.barplot(data=plot_df,x='TICKER',y='VOL')
# f, axs = plt.subplots(2, 2, figsize=(12, 8), gridspec_kw=dict(width_ratios=[4, 4]))
# sns.barplot(data=plot_df,x='TICKER',y='RSI',ax=axs[0,0])
# sns.barplot(data=plot_df,x='TICKER',y='VOL',ax=axs[0,1])
# sns.barplot(data=plot_df,x='TICKER',y='%CHG',ax=axs[1,0])
# sns.barplot(data=plot_df,x='TICKER',y='AVG10ATR',ax=axs[1,1])
# f.tight_layout()
# #f.savefig("test.pdf")


In [ ]:
f,axs=plt.subplots(1,2,figsize=(30,10))
sns.barplot(data=df,x='Date',y='Volume',ax=axs[0])
sns.lineplot(data=df,x='Date',y='Adj Close',ax=axs[1])
# axs[0].tick_params(axis='x', rotation=30)
# axs[1].tick_params(axis='x', rotation=30)
for i,j in enumerate(axs.flat):
    axs[i].tick_params(axis='x', rotation=30)
    

In [ ]:
summary_df=pd.DataFrame()
summary_df['RSI']=temp_df.sort_values(by=['RSI'],ascending=False).index
summary_df['VOL']=temp_df.sort_values(by=['VOL'],ascending=False).index
summary_df['CP']=temp_df.sort_values(by=['CP'],ascending=False).index
summary_df['+CHG']=temp_df.sort_values(by=['%CHG'],ascending=False).index
summary_df['ATR']=temp_df.sort_values(by=['AVG10ATR'],ascending=False).index


# def insert_columns(values_by,given_df=temp_df):
#     temp=given_df.sort_values(by=[values_by])
#     summary_df[values_by]=given_df['TICKER']
#     return summary_df

# test=insert_columns("CP");
# #insert_columns("RSI")
# test
# temp_df.set_index('TICKER')
summary_df=change_index(summary_df)
summary_df

In [ ]:
for c,i in enumerate(temp_df['TICKER'].tolist()):
    sector=temp_df.loc[c+1,"SECTOR"];industry=temp_df.loc[c+1,"SUBINDUSTRY"];cp=temp_df.loc[c+1,"CP"];name=temp_df.loc[c+1,"NAME"]

    print_info(c,sector,industry,name,i,cp)
    plot_with_mpl(i,csv_source,50,'default')

In [ ]:
#temp_df["SECTOR"]=st_df["SECTOR"];temp_df["SUBINDUSTRY"]="N/A";temp_df["NAME"]="N/A"
#temp_df=temp_df[(temp_df['HH']==True)&(temp_df['HL']==True)]
temp_df_rsi=temp_df.sort_values(by=['RSI'],ascending=False)
temp_df_rsi=change_index(temp_df_rsi)
temp_df_rsi.head(30)

In [ ]:
print("RSI\n")
for c,i in enumerate(temp_df_rsi['TICKER'].tolist()):
    sector=temp_df_rsi.loc[c+1,"SECTOR"];industry=temp_df_rsi.loc[c+1,"SUBINDUSTRY"];cp=temp_df_rsi.loc[c+1,"CP"];name=temp_df_rsi.loc[c+1,"NAME"]

    print_info(c,sector,industry,name,i,cp)
    plot_with_mpl(i,csv_source,50,'default')

In [ ]:
#temp_df["SECTOR"]=st_df["SECTOR"];temp_df["SUBINDUSTRY"]="N/A";temp_df["NAME"]="N/A"
#temp_df=temp_df[(temp_df['HH']==True)&(temp_df['HL']==True)]
temp_df=temp_df.sort_values(by=['%CHG'],ascending=False)
temp_df=change_index(temp_df)
temp_df.head(30)

In [ ]:
for c,i in enumerate(temp_df['TICKER'].tolist()):
    sector=temp_df.loc[c+1,"SECTOR"];industry=temp_df.loc[c+1,"SUBINDUSTRY"];cp=temp_df.loc[c+1,"CP"];name=temp_df.loc[c+1,"NAME"]

    print_info(c,sector,industry,name,i,cp)
    plot_with_mpl(i,csv_source,50,'default')

In [ ]:
#test file
dff=pd.read_csv(save+"XLF.csv",parse_dates=['Date'])
dff.set_index('Date',inplace=True)
dff.tail()

In [ ]:
#test file
dff1=pd.read_csv(save+"XLF.csv",parse_dates=['Date'],index_col='Date')
#dff.set_index('Date',inplace=True)
dff1.tail()
#dff1.info()

In [ ]:
# dff['Date']=pd.to_datetime(dff.Date)
# dff.set_index('Date')

#dff['Date'].dt.day_name()
#dff.index.min(),df.index.max()
#dff['2022-01-01':]
#df_close=dff1['Close'].resample('W').agg(['min','max'])
#df_agg=dff1.resample('W').agg(['min','max'])
df_agg=dff1.resample('BM').agg({'Volume':'mean'})
df_agg['Volume']=round(df_agg['Volume']/10**6,2)
df_agg

In [ ]:
%matplotlib inline
dff2=dff1.resample('W').last()#.agg()
#df_agg['Volume']=round(df_agg['Volume']/10**6,2)
#df_agg
dff2[-4:]
plt.plot(dff2.index,dff2.Close,label='w')

In [ ]:
#%matplotlib qt
#matplotlib inline
plt.figure(figsize=(16,8))
dff3=dff1.resample('BM').last()#.agg()
#df_agg['Volume']=round(df_agg['Volume']/10**6,2)
#df_agg
dff3[-4:]
plt.plot(dff3.index,dff3.Close,'r--',label='m')
plt.plot(dff2.index,dff2.Close,'b-.',label='w')
plt.plot(dff1.index,dff1.Close,'g',label='d')
plt.xlim('2022-05','2022-08-30')

plt.legend()

In [ ]:
#%matplotlib qt
%matplotlib inline
plt.style.use('seaborn')
fig, axs = plt.subplots(3, 1,figsize=(6,4))
axs[0].plot(dff1.tail(20).index,dff1.tail(20).Close,'r',label='d')
#axs[0, 0].set_title('Axis [0, 0]')
dff2=dff2.tail(20)
axs[1].plot(dff2.index,dff2.Close,'b',label='w')
axs[2].plot(dff3.index,dff3.Close,'g',label='m')
fig.legend()
plt.tight_layout()
# axs[0, 1].set_title('Axis [0, 1]')
# axs[1, 0].plot(x, -y, 'tab:green')
# axs[1, 0].set_title('Axis [1, 0]')
# axs[1, 1].plot(x, -y, 'tab:red')
# axs[1, 1].set_title('Axis [1, 1]')

In [ ]:
%matplotlib inline
fig = mpf.figure(figsize=(12,8),style='yahoo')

In [ ]:
ax1 = fig.add_subplot(3,1,1)
ax2 = fig.add_subplot(3,1,2)
ax3 = fig.add_subplot(3,1,3)
mpf.plot(dff1,ax=ax1,type='candle',axtitle="daily")
mpf.plot(dff2,ax=ax2,type='candle',axtitle="weekly")
mpf.plot(dff3,ax=ax3,type='candle',axtitle="monthly")
fig

In [ ]:
apds = [mpf.make_addplot(exp12,color='lime'),
        mpf.make_addplot(exp26,color='c'),
        mpf.make_addplot(histogram,type='bar',width=0.7,panel=1,
                         color='dimgray',alpha=1,secondary_y=False),
        mpf.make_addplot(macd,panel=1,color='fuchsia',secondary_y=True),
        mpf.make_addplot(signal,panel=1,color='b',secondary_y=True),
       ]

mpf.plot(df,type='candle',addplot=apds,figscale=1.1,figratio=(8,5),title='\nMACD',
         style='blueskies',volume=True,volume_panel=2,panel_ratios=(6,3,2))


In [ ]:
df_w=dff1[dff1.index.day_name()=='Friday']
df_w.tail(5)

In [ ]:
%matplotlib inline
df_agg.plot()

In [ ]:
#group by week 
# df_week=dff.resample('BM',on='Date')
# df_week.min()


In [ ]:
#group by week 
# df_week=dff.resample('BM',on='Date')
# df_week.max()
